In [33]:
from keras.applications.resnet50 import ResNet50, preprocess_input

In [7]:
HEIGHT = 197
WIDTH = 197

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

94658560/94653016 [==============================] - 8s 0us/step


In [9]:
import numpy as np
import json
import os
post_building_files = sorted(os.listdir('data/train_images/post_buildings'))

json_source = 'data/train_labels/post_labels/'
post_hurricane_jsons = sorted([f for f in os.listdir(json_source)])

damage_labels = []
for image_json in post_hurricane_jsons:
    with open (json_source + image_json) as f:
        json_data = json.load(f)
        
    for building in json_data['features']['xy']:
        damage_labels.append(building['properties']['subtype'])

In [11]:
from PIL import Image

Images = []
for f in post_building_files:
    Images.append(Image.open('data/train_images/post_buildings/' + f).convert('RGB'))
    
X_post = []
for I in Images:
    X_post.append(np.asarray(I))
    
X_post = np.array(X_post)
X_post.shape

(66092, 50, 50, 3)

In [31]:
X_post_padded = [np.pad(img, ((73, 74), (73, 74), (0,0)), 'constant', constant_values=(255,)) for img in X_post] 

In [32]:
X_post_padded = np.array(X_post_padded)
X_post_padded.shape

(66092, 197, 197, 3)

In [36]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

## remove the FC layers
dropout = 0.5
num_classes = 2
FC = [1024, 1024]

for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = Flatten()(x)

for fc in FC:
    x = Dense(fc, activation='relu')(x)
    x = Dropout(dropout)(x)

# new softmax layer:
predictions = Dense(num_classes, activation='softmax')(x)

new_model = Model(inputs=base_model.input, outputs=predictions)
